In [1]:
import pandas as pd
import numpy as np
import ast
from sklearn.feature_extraction.text import CountVectorizer
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
limit = WordNetLemmatizer()

nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\SOHAM\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\SOHAM\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\SOHAM\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:
file_path1 = r"C:\Users\SOHAM\Desktop\movies\tmdb_5000_credits.csv"
data1 = pd.read_csv(file_path1)
file_path2 = r"C:\Users\SOHAM\Desktop\movies\tmdb_5000_movies.csv"
data2 = pd.read_csv(file_path2)

In [4]:
data1 = data1[['movie_id', 'cast', 'crew']]


In [5]:
data2.rename(columns={'id': 'movie_id'}, inplace=True)


In [6]:
movies_data = pd.merge(data1,data2,on = "movie_id",how = "inner")

In [7]:
movies = movies_data.drop(['original_language', 'original_title','popularity','revenue','runtime','status','vote_average','vote_count','budget','homepage','spoken_languages','tagline'	],axis=1)

In [8]:
#movies.isnull().sum()

In [9]:
movies.dropna(inplace = True)

In [10]:
def change (ndarray,string ):
    genre = []
    for item in ast.literal_eval(ndarray):
      genre.append(item[string])

    return genre        
     

In [11]:
movies['genres'] =  movies['genres'].apply(lambda n:change(n,'name'))
movies['cast'] =  movies['cast'].apply(lambda n:change(n,'name')).apply(lambda x:x[:1])
movies['keywords'] =  movies['keywords'].apply(lambda n:change(n,'name'))
movies['production_companies'] =  movies['production_companies'].apply(lambda n:change(n,'name'))
movies['production_countries'] =  movies['production_countries'].apply(lambda n:change(n,'name'))
movies['crew'] = movies['crew'].apply(lambda n: [d['name'] for d in ast.literal_eval(n) if d['department'] == 'Directing'])

In [12]:
#movies.head()

In [13]:
movies['merge1'] = movies['genres'] + movies['cast'] + movies['keywords'] + movies['production_companies'] + movies['production_countries'] + movies['crew']

In [14]:
movies['merge1'] = movies['merge1'].apply(lambda ndarr: " ".join(st.replace(" " ,"") for st in ndarr))

In [15]:
movies['tags'] = movies['merge1'] + movies['overview']

In [16]:
movies = movies[['movie_id','release_date','title','tags']]

In [17]:
movies_full = movies.copy()

In [18]:
movies = movies[:1800]

In [19]:
movies.shape

(1800, 4)

In [20]:
movies['tags'] = movies['tags'].apply(lambda n:" ".join([limit.lemmatize(w) for w in word_tokenize(n)]))

In [21]:
total_words = movies['tags'].apply(lambda x: len(word_tokenize(str(x)))).sum()
total_words

140741

In [22]:
#CountVectorizer().get_params()

In [23]:
Vectorizer = CountVectorizer(max_features = 3500,stop_words = 'english',strip_accents ='ascii')

In [24]:
vect = Vectorizer.fit_transform(movies['tags']).toarray()

In [25]:
print(Vectorizer.get_feature_names_out())

['000' '007' '10' ... 'zeus' 'zombie' 'zoo']


In [26]:
cosine_vect = cosine_similarity(vect)

In [27]:
def find (movie):
    index = movies.index[movies['title'] == movie]
    c_v = cosine_vect[index][0]
    c_v[index] = 0
    ind = []
    for i in np.arange(10):
      pos = np.argmax(c_v)
      c_v[pos] = 0
      ind.append(pos)
        
    return titles(ind)
   

In [28]:
def titles (lst):
    df = pd.DataFrame(columns =['title','release_date'])
    for i in lst:
        row = movies.loc[[i],['title','release_date']]
        df =pd.concat([df, row],axis = 0,ignore_index = True)

    print(df)    

In [29]:
movies.to_csv('movie.csv')

In [30]:
c_v_df =pd.DataFrame(cosine_vect)

In [31]:
c_v_df.to_csv('cosine_vect.csv',index = False)